#### A linear model to discern flowers using Tensorflow
We'll build a linear model to recognize different kinds of flowers using a canned estimator.

We will use Google's Tensorflow: an open source machine learning tool for everyone.

The types of flowers we would like to distunguish is Iris Versicolour, Iris Virginica, Iris Setosa. 

Here are the Iris flowers we'd want our model to train on

* Iris Virginica
![Iris Virginica](images/178px-Iris_virginica.jpg)

* Iris Versicolor
![Iris_versicolor](images/193px-Iris_versicolor_3.jpg)

* Iris Setosa
![Iris Kosaciec](images/109px-Kosaciec_szczecinkowaty_Iris_setosa.jpg)

The data columns we have are:
* sepal length in cm
* sepal width in cm
* petal length in cm
* petal width in cm



In [144]:
# Step 1: split the data into training and testing data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

actual_data = pd.read_csv("iris.csv")
data = pd.read_csv("iris.csv", skiprows=[0], header=None)
labels_data = pd.read_csv("iris.csv", usecols=[4], skiprows=[0], header=None)
labels = np.unique(np.array(labels_data, 'str'))

for index, row in data.iterrows():
    if row[4] == labels[0]:
        data.loc[index, 4] = 0
    elif row[4] == labels[1]:
        data.loc[index, 4] = 1
    else:
        data.loc[index, 4] = 2

# shuffle the newly formatted data
data = data.sample(frac=1).reset_index(drop=True)
# split that data, 80-20
y = actual_data.species

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)
print (X_test)
# write the split data to csv
# but first create a custom header
# omit the last column since that's the one we wish to predict
X_train_header = list((len(X_train.index), len(X_train.columns) - 1))  + list(labels)
X_test_header = list((len(X_test.index), len(X_test.columns) - 1)) + list(labels)
print (X_train_header)
print (X_test_header)
X_train.to_csv("iris_training.csv", index=False, index_label=False, header=X_train_header)
X_test.to_csv("iris_test.csv", index=False, index_label=False, header=X_test_header)

       0    1    2    3  4
71   5.5  2.3  4.0  1.3  1
140  6.7  3.0  5.2  2.3  2
23   6.0  2.2  4.0  1.0  1
76   6.8  3.2  5.9  2.3  2
131  4.8  3.0  1.4  0.3  0
58   4.8  3.4  1.9  0.2  0
46   6.5  3.0  5.8  2.2  2
28   6.0  2.9  4.5  1.5  1
80   5.8  2.7  5.1  1.9  2
149  5.6  2.9  3.6  1.3  1
145  6.6  2.9  4.6  1.3  1
20   5.6  2.8  4.9  2.0  2
1    6.5  3.0  5.2  2.0  2
36   6.9  3.1  5.4  2.1  2
136  6.6  3.0  4.4  1.4  1
111  6.4  2.8  5.6  2.2  2
56   4.9  3.1  1.5  0.1  0
120  6.7  3.1  4.4  1.4  1
144  5.2  4.1  1.5  0.1  0
98   7.7  2.8  6.7  2.0  2
81   5.0  3.5  1.6  0.6  0
133  6.2  2.2  4.5  1.5  1
117  7.2  3.2  6.0  1.8  2
85   5.7  3.0  4.2  1.2  1
65   5.1  2.5  3.0  1.1  1
26   6.2  2.9  4.3  1.3  1
87   5.0  3.4  1.6  0.4  0
132  5.8  2.7  4.1  1.0  1
130  5.9  3.0  4.2  1.5  1
8    6.3  3.4  5.6  2.4  2
[120, 4, 'setosa', 'versicolor', 'virginica']
[30, 4, 'setosa', 'versicolor', 'virginica']


In [145]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base

# Data files containing our sepal and petal features
IRIS_TRAINING = "iris_training.csv"
IRIS_TEST = "iris_test.csv"

training_set = base.load_csv_with_header(
    filename=IRIS_TRAINING, 
    features_dtype=np.float32, 
    target_dtype=np.int)

test_set = base.load_csv_with_header(
    filename=IRIS_TEST,
    features_dtype=np.float32,
    target_dtype=np.int)

print(test_set.target)




[1 2 1 2 0 0 2 1 2 1 1 2 2 2 1 2 0 1 0 2 0 1 2 1 1 1 0 1 1 2]


In [147]:
# Specify that all feature columns have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

Then we build the 3 layer Deep Nueral Network

In [149]:
classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=featured_columns,
    hidden_units=[10, 20, 10],
    n_classes=3,
    model_dir="/tmp/iris_model"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1f7e7f98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/iris_model'}


Next, we define the training inputs

In [150]:
def training_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    
    return x,y

Then next step is to fit the model with the training data. Fitting is where the model is trained.

In [151]:
# fit the classifier with the training data
classifier.fit(input_fn=training_inputs, steps=2000)

Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 1.5912925, step = 1
INFO:tensorflow:global_step/sec: 827.676
INFO:tensorflow:loss = 0.28244945, step = 101 (0.123 sec)
INFO:tensorflow:global_step/sec: 990.599
INFO:tensorflow:loss = 0.12283801, step = 201 (0.101 sec)
INFO:tensorflow:global_step/sec: 921.432
INFO:tensorflow:loss = 0.084290884, step = 301 (0.108 sec)
INFO:tensorflow:global_step/sec: 832.59
INFO:tensorflow:loss = 0.07436801, step = 401 (0.124 sec)
INFO:tensorflow:global_step/sec: 857.154
INFO:tensorflow:loss = 0.06916182, step = 501 (0.115 sec)
INFO:tensorflow:global_step/sec: 964.682
INFO:tensorflow:loss = 0.065359734, step = 601 (0.101 sec)
INFO:tensorflow:global_step/sec: 1063.21
INFO:tensorflow:loss = 0.06362448, step = 701 (0.095 sec)
INFO:tensorflow:global_step/sec: 840.237
INFO:tensorflow:loss = 0.061846804,

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x1c1f8f7e10>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x1160d4d90>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [152]:
# Define test inputs
def test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    
    return x, y

After training, we evaluate the accuracy of our trained model. 
We do this using the evaluate method. It takes in the test input data and target to build its input data pipeline. After measuring the model's accuracy, it returns a dictionary containing the results.

In [156]:
# evaluate the classifier's accuracy
accuracy_score = classifier.evaluate(input_fn=test_inputs, steps=1)['accuracy']
print ("Accuracy score", accuracy_score)

INFO:tensorflow:Starting evaluation at 2018-03-03-16:42:26
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-03-03-16:42:26
INFO:tensorflow:Saving dict for global step 2000: accuracy = 1.0, global_step = 2000, loss = 0.02180128
Accuracy score 1.0


Time to see if our model can predict the type of Iris flower given a new flower sample. 

In [157]:
# Classify two new flower samples.
def new_flower_samples():
    return np.array(
        [[6.4, 3.2, 4.5, 1.5],
        [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)


In [ ]:
# Predict the type of Iris flower
prediction = classifier.predict(input_fn=new_flower_samples)